# Feeding data to a production model

This notebook is part of the [watsonx.governance Level 4 Proof of Experience (PoX) hands-on lab](watsonx.governance Level 4 Proof of Experience (PoX) hands-on lab). It will query a deployed production model

In [18]:
USERNAME = "complianceofficer"
PASSWORD = "passw0rd"
CPD_URL = "https://cpd-cpd.apps.___________.cloud.techzone.ibm.com"
INSTANCE_ID = "openshift"
VERSION = "4.8"
DEPLOYMENT_ID = ""
SPACE_GUID = ""
RECORDS_TO_SCORE = 100

### Authenticate with the watsonx service running in the environment

In [19]:
from ibm_watsonx_ai import APIClient

wml_credentials = {
    "username": USERNAME,
    "password": PASSWORD,
    "url": CPD_URL,
    "instance_id": 'openshift',
    "version": '4.8'
}

client = APIClient(wml_credentials)

### Set the client to use the deployment space for the production model

In [4]:
client.set.default_space(SPACE_GUID)

'SUCCESS'

### Import data to be scored

In [5]:
import pandas as pd

feed_dataframe = pd.read_csv("https://raw.githubusercontent.com/CloudPak-Outcomes/Outcomes-Projects/main/watsonx-governance-l4/data/hiring_training_data.csv")
feed_dataframe.head()

,HIRED,Age,BusinessTravel,Education,RelevantEducationLevel,JobLevel,MaritalStatus,NumCompaniesWorked,OverTime,InterviewScore,ResumeScore,RequestedBenefits,TotalWorkingYears,PreferredSkills,YearsAtCurrentCompany,RelevantExperience,JobType,SalaryExpectation,IsFemale
0,1,33,2,3,3,0,0,1,1,2,2,0,8,2,8.0,7,1,0,1
1,1,32,2,2,3,0,1,0,0,3,2,0,8,1,7.0,7,0,1,0
2,1,35,1,1,4,0,0,0,0,1,2,1,6,1,5.0,4,1,1,0
3,1,29,1,3,4,2,2,1,0,0,2,1,10,0,10.0,9,1,0,1
4,1,34,1,3,3,2,1,0,0,1,2,0,13,2,12.0,6,1,2,1


### Remove the label column from the data

In [6]:
feed_dataframe.drop(['HIRED'], axis=1, inplace=True)

### Identify the features and values, and construct the payload data

In [8]:
features = feed_dataframe.columns.values
values = feed_dataframe.sample(n=RECORDS_TO_SCORE).round().astype(int).values

scoring_payload = {client.deployments.ScoringMetaNames.INPUT_DATA:
    [{'fields':
        features,
        'values': values
    }]
}

### Send the scoring payload to the model for scoring

In [16]:
predictions = client.deployments.score(DEPLOYMENT_ID, scoring_payload)

### Display the model predictions

If you have configured monitoring for the model, the input and output data here will be automatically recorded in the watsonx monitoring service datamart, and you will be able to calculate drift and fairness using the data.

In [17]:
predictions

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [[0.0, [0.96, 0.04]], [0.0, [1.0, 0.0]], [1.0, [0.0, 1.0]]]}]}